# AI Language Model Interface Documentation

## Introduction
AL simplifies the complexity of large language models into a simple, open-source solution. One intuitive interface. Multiple AI powerhouses. Ollama, Anthropic, OpenAI - seamlessly integrated into all python environments, you can use it even on 1970 terminals if youd like that sort of a thing.
This framework simplifies interactions with large language models through a consistent, open-source interface. It provides seamless integration with multiple AI providers (Ollama, Anthropic, OpenAI) in any Python environment, from modern IDEs to basic terminals.

The interface preserves the complete interaction history, captures code snippets, and integrates smoothly with Jupyter notebooks. This enables easy sharing, versioning, and automation of AI workflows. The saved sessions serve as valuable resources for:
- Model fine-tuning
- Workflow iteration
- Learning and training
- Pattern analysis
- Response comparison across models

Key Features:
* Simple markdown/ASCII interface
* Flexible model selection
* Conversation history management
* Code snippet extraction and management
* Jupyter notebook integration

## Why Use This Interface?

1. **Unified API**: Consistent interaction patterns across different AI models
2. **Modular Design**: Easy to extend and customize for new AI services
3. **Built-in History**: Automatic conversation tracking and management
4. **Code Management**: Automatic extraction and organization of code snippets
5. **Notebook Support**: Native integration with Jupyter environments

## Class Structure

### Base Class: AILanguageModel

The `AILanguageModel` serves as the abstract base class defining the core interface:

```python
class AILanguageModel:
    def __init__(self, model: str):
        # Initialize model configuration
    
    def send_message(self, message: str) -> None:
        # Send message and handle response
    
    def save_code(self, index: Optional[int] = None) -> None:
        # Save code snippet to clipboard
```

### Implementation Classes

Each AI service has its own implementation:

```python
class OllamaModel(AILanguageModel):
    def __init__(self, model: str = "llama3.1"):
        # Ollama-specific initialization

class ClaudeModel(AILanguageModel):
    def __init__(self, model: str, api_key: str):
        # Claude-specific initialization

class ChatGPTModel(AILanguageModel):
    def __init__(self, model: str, api_key: str):
        # ChatGPT-specific initialization
```

## Usage Examples

### Basic Usage

```python
# Using Ollama
llm = OllamaModel("llama3.1")
llm.send_message("Write a quicksort implementation")

# Using Claude
claude = ClaudeModel("claude-3", "your-api-key")
claude.send_message("Explain quantum computing")

# Using ChatGPT
gpt = ChatGPTModel("gpt-4", "your-api-key")
gpt.send_message("Create a React component")
```

### Managing Code Snippets

```python
# Save the most recent code snippet
llm.save_code()

# Save a specific code snippet by index
llm.save_code(1)  # Saves the second-most-recent snippet
```

## Extending the System

To add support for a new AI service:

1. Create a new class inheriting from `AILanguageModel`:

```python
class NewAIModel(AILanguageModel):
    """
    Implementation for a new AI service with streaming support
    
    Args:
        model (str): Name or identifier of the model to use
        api_key (str): API key for the service
        stream (bool): Whether to stream responses (default: True)
    """
    
    def __init__(self, model: str, api_key: str, stream: bool = True):
        super().__init__(model, stream)
        # Initialize your API client
        self.client = YourAPIClient(api_key)

    def _send_to_model(self, messages: List[Dict]) -> Union[str, Generator[str, None, None]]:
        """
        Send messages to the model and get response
        
        Args:
            messages: List of conversation messages
            
        Returns:
            Union[str, Generator[str, None, None]]: Either complete response string or
                                                   generator yielding response chunks
        """
        if self.stream:
            return self._stream_response(messages)
        else:
            # Implement non-streaming API call
            response = self.client.send_message(messages)
            return response.text
    
    def _stream_response(self, messages: List[Dict]) -> Generator[str, None, None]:
        """
        Stream response from the model
        
        Args:
            messages: List of conversation messages
            
        Yields:
            str: Response text chunks
        """
        # Implement streaming API call
        for chunk in self.client.stream_message(messages):
            if chunk.has_content:
                yield chunk.text
```

2. Key methods to implement:
   - `__init__`: Set up your model and API client
   - `_send_to_model`: Handle the actual API communication
   - `_stream_response`: For streaming support

3. Optional overrides:
   - `_extract_code_blocks`: If your model returns code in a different format
   - `_add_to_history`: If you need custom history management
   - 
4. Response Processing:
   - For streaming: ```yield``` text chunks ```as``` they arrive
   - For non-streaming: ```return``` complete response text
   - Ensure consistent markdown formatting

## Core Components

### Message Handling
- `send_message`: Primary method for sending user input
- `_send_to_model`: Model-specific API communication
- `_add_to_history`: Conversation history management

### Code Management
- `save_code`: Save code snippets to clipboard
- `_extract_code_blocks`: Parse code from responses

### History Management
- `conversation_history`: Stores all interactions
- `code_snippets`: Maintains extracted code blocks

## Best Practices

1. **Model Selection**:
   - Choose the appropriate model class for your use case
   - Consider rate limits and API costs

2. **Error Handling**:
   - Implement proper API error handling
   - Monitor API quota usage

3. **History Management**:
   - Regularly save important conversations
   - Clear history for sensitive information

4. **Extension Development**:
   - Follow the established pattern for new models
   - Maintain consistent interface behavior

## Conclusion

This framework provides a robust foundation for AI model interactions. Its modular design allows easy integration of new services while maintaining a consistent interface. The built-in features for history management and code handling make it particularly valuable for development workflows and educational contexts.

In [165]:
from typing import Optional, List, Dict, Generator, Union
from IPython.display import Markdown, display
from datetime import datetime
import pyperclip
from abc import ABC, abstractmethod
import sys
from IPython.display import clear_output

class AILanguageModel(ABC):
    """Base class for AI language model interactions with streaming support"""
    
    def __init__(self, model: str, stream: bool = True):
        self.model = model
        self.stream = stream
        self.conversation_history = []
        self.code_snippets = []
        self.current_snippet_index = 0
        self._current_response = ""

    def send_message(self, message: str) -> None:
        """Send a message to the AI model and display its response"""
        # Record user message
        self._add_to_history("user", message)
        
        # Get response
        if self.stream:
            self._current_response = ""
            for chunk in self._send_to_model(self.conversation_history):
                self._current_response += chunk
                # Clear previous output and display updated response
                clear_output(wait=True)
                display(Markdown(self._current_response))
            
            response = self._current_response
        else:
            response = self._send_to_model(self.conversation_history)
            display(Markdown(response))
        
        # Process response after completion
        self._extract_code_blocks(response)
        self._add_to_history("assistant", response)

    def save_code(self, index: Optional[int] = None) -> None:
        """Save a code snippet to clipboard"""
        index = index if index is not None else self.current_snippet_index
        
        if 0 <= index < len(self.code_snippets):
            code = self.code_snippets[index]
            pyperclip.copy(code)
            print(f"Saved code snippet {index} to clipboard:")
            print(code)
        else:
            print("Invalid code snippet index")

    def _add_to_history(self, role: str, content: str) -> None:
        """Add a message to conversation history"""
        self.conversation_history.append({
            "role": role,
            "content": content,
            "timestamp": datetime.now().isoformat()
        })

    def _extract_code_blocks(self, text: str) -> None:
        """Extract code blocks from markdown-formatted text"""
        parts = text.split('```')
        for i, part in enumerate(parts):
            if i % 2 == 1:  # Code block
                lines = part.splitlines()
                if lines:
                    # Remove language identifier from first line
                    code = '\n'.join(lines[1:]).strip()
                    self.code_snippets.insert(0, code)

    @abstractmethod
    def _send_to_model(self, messages: List[Dict]) -> Union[str, Generator[str, None, None]]:
        """
        Send messages to model and get response
        
        Args:
            messages: List of message dictionaries
            
        Returns:
            Union[str, Generator[str, None, None]]: Either complete response string or
                                                   generator yielding response chunks
        """
        pass

# Jupyter Notebook Integration Guide

## Overview

AL doesn't just fetch answers; it preserves your learning journey. Every interaction, every spark of inspiration, is captured in Jupyter notebooks. Share, version, and automate your AI workflows with ease.
The Jupyter notebook integration enables seamless conversation management through `.ipynb` files. This functionality serves two primary purposes:
1. Creating fine-tuning datasets by editing and refining conversations
2. Building template conversations for automated responses

## Features

### Conversation Export
- Preserves complete conversation history
- Maintains markdown formatting
- Separates code blocks into executable cells
- Retains role information (user/assistant)

### Conversation Import
- Loads existing conversations
- Reconstructs conversation history
- Extracts code snippets automatically
- Validates conversation structure

## Usage Examples

### Exporting Conversations

```python
# Create and use your model
model = AILanguageModel("your-model")
model.send_message("Hello, can you explain Python generators?")

# Export the conversation
model.export_to_notebook("python_explanation.ipynb")
```

### Importing and Modifying Conversations

```python
# Load an existing conversation
model = AILanguageModel("your-model")
model.import_from_notebook("template_responses.ipynb")

# Continue the conversation
model.send_message("Can you elaborate on the last point?")
```

## Creating Fine-tuning Datasets

The notebook format allows you to create and edit training data efficiently:

1. **Record Initial Conversations**
   ```python
   model.send_message("Your query")
   model.export_to_notebook("training_data.ipynb")
   ```

2. **Edit in Jupyter Lab**
   - Open the notebook in Jupyter Lab
   - Modify responses for better quality
   - Add alternative phrasings
   - Remove irrelevant parts
   - Add more context or examples

3. **Import Modified Conversation**
   ```python
   model.import_from_notebook("improved_training_data.ipynb")
   ```

## Building Template Conversations

Create reusable conversation templates for common scenarios:

1. **Create Base Template**
   ```python
   # Record template conversation
   model.send_message("Template query with placeholders")
   model.export_to_notebook("template.ipynb")
   ```

2. **Customize in Jupyter Lab**
   - Add placeholder markers
   - Include alternative responses
   - Add conditional sections
   - Document usage instructions

3. **Use as Response Template**
   ```python
   # Load template and customize
   model.import_from_notebook("customer_support_template.ipynb")
   ```

## Notebook Structure

### Cell Types
1. **Role Headers**
   - Markdown cells starting with `## User` or `## Assistant`
   - Indicate the speaker for following content

2. **Content Cells**
   - Markdown cells for regular text
   - Code cells for executable code
   - Preserves formatting and structure

## Tips for Effective Use

1. **Fine-tuning**
   - Start with real conversations
   - Edit for clarity and correctness
   - Add alternative phrasings
   - Include edge cases

2. **Templates**
   - Use clear placeholder markers
   - Include usage instructions
   - Document customization points
   - Provide examples

3. **Version Control**
   - Keep backup copies
   - Track changes
   - Document modifications
   - Test before deployment

In [139]:
from typing import Optional, List, Dict
import nbformat as nbf
from datetime import datetime
from IPython.display import Markdown

class AILanguageModel(AILanguageModel):
    """AILanguageModel with Jupyter notebook support for saving and loading conversations"""
    
    def export_to_notebook(self, filename: str) -> None:
        """
        Save conversation history to a Jupyter notebook
        
        Args:
            filename (str): Target notebook filename (.ipynb extension optional)
        """
        if not filename.endswith('.ipynb'):
            filename += '.ipynb'
            
        notebook = nbf.v4.new_notebook()
        cells = []
        
        for entry in self.conversation_history:
            # Add role header
            cells.append(nbf.v4.new_markdown_cell(
                f"## {entry['role'].capitalize()}"
            ))
            
            # Split content into code and markdown
            parts = entry['content'].split('```')
            
            for i, part in enumerate(parts):
                if i % 2 == 0 and part.strip():  # Markdown content
                    cells.append(nbf.v4.new_markdown_cell(part.strip()))
                elif i % 2 == 1:  # Code content
                    lines = part.splitlines()
                    if lines:
                        code = '\n'.join(lines[1:]).strip()  # Skip language identifier
                        cells.append(nbf.v4.new_code_cell(code))
                    
        notebook['cells'] = cells
        
        with open(filename, 'w', encoding='utf-8') as f:
            nbf.write(notebook, f)
            
        print(f"Conversation exported to {filename}")
    
    def import_from_notebook(self, filename: str) -> None:
        """
        Load conversation history from a Jupyter notebook
        
        Args:
            filename (str): Source notebook filename (.ipynb extension optional)
        """
        if not filename.endswith('.ipynb'):
            filename += '.ipynb'
            
        # Read notebook
        with open(filename, 'r', encoding='utf-8') as f:
            notebook = nbf.read(f, as_version=4)
        
        # Clear existing history
        self.conversation_history = []
        self.code_snippets = []
        
        current_entry = None
        
        # Process cells
        for cell in notebook.cells:
            source = cell['source']
            
            if cell['cell_type'] == 'markdown' and source.startswith('## '):
                # Save previous entry if exists
                if current_entry:
                    self.conversation_history.append(current_entry)
                
                # Start new entry
                role = source[3:].lower().strip()
                current_entry = {
                    'role': role,
                    'content': '',
                    'timestamp': datetime.now().isoformat()
                }
            
            elif current_entry:
                # Add content to current entry
                if cell['cell_type'] == 'code':
                    if current_entry['role'] == 'assistant':
                        self.code_snippets.insert(0, source)
                    current_entry['content'] += f'```python\n{source}\n```\n'
                else:
                    current_entry['content'] += f'{source}\n'
        
        # Add final entry if exists
        if current_entry:
            self.conversation_history.append(current_entry)
            
        print(f"Conversation imported from {filename}")
        print(f"Loaded {len(self.conversation_history)} messages and {len(self.code_snippets)} code snippets")

In [157]:
from typing import List, Dict, Union, Generator
import ollama

class OllamaModel(AILanguageModel):
    """
    Ollama-specific implementation with streaming support
    
    Args:
        model (str): Name of the Ollama model to use
        stream (bool): Whether to stream responses (default: True)
    """
    
    def __init__(self, model: str = "llama3.1", stream: bool = True):
        super().__init__(model, stream)
        self.client = ollama

    def _send_to_model(self, messages: List[Dict]) -> Union[str, Generator[str, None, None]]:
        if self.stream:
            return self._stream_response(messages)
        else:
            response = self.client.chat(
                model=self.model,
                messages=messages
            )
            return response['message']['content']
    
    def _stream_response(self, messages: List[Dict]) -> Generator[str, None, None]:
        #Stream response from Ollama model
        for chunk in self.client.chat(
            model=self.model,
            messages=messages,
            stream=True
        ):
            if 'message' in chunk and 'content' in chunk['message']:
                yield chunk['message']['content']

Intrface for Antropic Claude, currently untested

In [ ]:
from typing import List, Dict, Union, Generator
from anthropic import Anthropic

class ClaudeModel(AILanguageModel):
    """
    Anthropic Claude-specific implementation with streaming support
    
    Args:
        model (str): Name of the Claude model to use
        api_key (str): Anthropic API key
        stream (bool): Whether to stream responses (default: True)
    """
    
    def __init__(self, model: str, api_key: str, stream: bool = True):
        super().__init__(model, stream)
        self.client = Anthropic(api_key=api_key)

    def _send_to_model(self, messages: List[Dict]) -> Union[str, Generator[str, None, None]]:
        if self.stream:
            return self._stream_response(messages)
        else:
            response = self.client.messages.create(
                model=self.model,
                messages=messages
            )
            return response.content[0].text
    
    def _stream_response(self, messages: List[Dict]) -> Generator[str, None, None]:
        #Stream response from Claude model
        with self.client.messages.create(
            model=self.model,
            messages=messages,
            stream=True
        ) as stream:
            for chunk in stream:
                if chunk.type == "content_block_delta":
                    yield chunk.delta.text
                elif chunk.type == "message_delta":
                    continue
                elif chunk.type == "error":
                    raise Exception(f"Error in stream: {chunk.error}")
                else:
                    continue

In [ ]:
from typing import List, Dict, Union, Generator
from openai import OpenAI

class ChatGPTModel(AILanguageModel):
    """
    OpenAI ChatGPT-specific implementation with streaming support
    
    Args:
        model (str): Name of the OpenAI model (e.g., "gpt-4", "gpt-3.5-turbo")
        api_key (str): OpenAI API key
        stream (bool): Whether to stream responses (default: True)
    """
    
    def __init__(self, model: str, api_key: str, stream: bool = True):
        super().__init__(model, stream)
        self.client = OpenAI(api_key=api_key)

    def _send_to_model(self, messages: List[Dict]) -> Union[str, Generator[str, None, None]]:
        if self.stream:
            return self._stream_response(messages)
        else:
            response = self.client.chat.completions.create(
                model=self.model,
                messages=messages
            )
            return response.choices[0].message.content
    
    def _stream_response(self, messages: List[Dict]) -> Generator[str, None, None]:
        #Stream response from OpenAI model
        stream = self.client.chat.completions.create(
            model=self.model,
            messages=messages,
            stream=True
        )
        
        for chunk in stream:
            if chunk.choices and chunk.choices[0].delta.content is not None:
                yield chunk.choices[0].delta.content

In [166]:
chat = OllamaModel()

In [167]:
chat.send_message("can you make me hello world in python")

Here is a simple "Hello, World!" program in Python:

**hello.py**
```python
print("Hello, World!")
```
To run this code, save it to a file called `hello.py` and then open a terminal or command prompt, navigate to the directory where you saved the file, and type:
```bash
python hello.py
```
This should print "Hello, World!" to the console.

Alternatively, if you're using an IDE (Integrated Development Environment) like PyCharm, Visual Studio Code, or Spyder, you can create a new Python project, paste this code into a file called `hello.py`, and run it from within the IDE.

In [170]:
chat.send_message("can you make me also code to generate a random number in the same language")

Here is an updated version of the "Hello, World!" program with an additional function that generates a random integer:

**random_hello.py**
```python
import random

def hello_world():
    print("Hello, World!")

def generate_random_number(min_val=1, max_val=100):
    return random.randint(min_val, max_val)

hello_world()
print("Random number:", generate_random_number())
```
This code defines two functions:

* `hello_world()`: prints "Hello, World!" to the console
* `generate_random_number()`: generates a random integer between `min_val` and `max_val` (defaulting to 1-100 if no arguments are provided)

The code then calls both functions in sequence. The output should look something like this:
```
Hello, World!
Random number: 42
```
Note that the actual random number will be different each time you run the program!

In [168]:
chat.conversation_history

[{'role': 'user',
  'content': 'can you make me hello world in python',
  'timestamp': '2024-12-20T12:09:29.287424'},
 {'role': 'assistant',
  'content': 'Here is a simple "Hello, World!" program in Python:\n\n**hello.py**\n```python\nprint("Hello, World!")\n```\nTo run this code, save it to a file called `hello.py` and then open a terminal or command prompt, navigate to the directory where you saved the file, and type:\n```bash\npython hello.py\n```\nThis should print "Hello, World!" to the console.\n\nAlternatively, if you\'re using an IDE (Integrated Development Environment) like PyCharm, Visual Studio Code, or Spyder, you can create a new Python project, paste this code into a file called `hello.py`, and run it from within the IDE.',
  'timestamp': '2024-12-20T12:09:32.223662'}]

In [169]:
chat.code_snippets

['python hello.py', 'print("Hello, World!")']